In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import gc
import sys

from sklearn.preprocessing import LabelEncoder
import scipy.stats as ss

if os.name=='nt':
    try:
        mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-8.1.0-posix-seh-rt_v6-rev0\\mingw64\\bin'
        os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']
    except:
        pass
    
import xgboost as xgb

%matplotlib inline

In [8]:
# Load data
train = pd.read_csv('all/sales_train.csv.gz')
test = pd.read_csv('all/test.csv.gz')
shop = pd.read_csv('all/shops-translated.csv')
item = pd.read_csv('all/item_category.csv')

In [9]:
test['date_block_num'] = train['date_block_num'].max()+1

In [10]:
train.drop(['item_price', 'date'], axis=1, inplace=True)

In [19]:
x = train.groupby(['date_block_num', 'shop_id', 'item_id'])['item_cnt_day'].sum().reset_index()

In [25]:
x_train = x.loc[x.date_block_num<33]
y_train = x_train['item_cnt_day']
x_train.drop('item_cnt_day', axis=1, inplace=True)

C:\Users\home\Anaconda3\envs\kaggle\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [27]:
x_test = x.loc[x.date_block_num==33]
y_test = x_test['item_cnt_day']
x_test.drop('item_cnt_day', axis=1, inplace=True)

C:\Users\home\Anaconda3\envs\kaggle\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [32]:
param = {'max_depth':10,
         'subsample':1,
         'min_child_weight':0.5,
         'eta':0.3, 
         'num_round':100,
         'seed':1,
         'silent':1,
         'eval_metric':'rmse'}

progress = dict()
xgbtrain = xgb.DMatrix(x_train, y_train)
watchlist  = [(xgbtrain,'train-rmse')]

bst = xgb.train(param, xgbtrain)
preds = bst.predict(xgb.DMatrix(x_test))
from sklearn.metrics import mean_squared_error 
rmse = np.sqrt(mean_squared_error(preds,y_test))
print(rmse)

13.53242169889497
